### Join points with all variables 

Create a gdf with all data from shp in directory

In [1]:
import glob
import geopandas as gpd
import pandas as pd

# Get a list of all shapefiles in the directory
shapefiles = glob.glob('*.shp')

# Initialize an empty list to store GeoDataFrames
gdfs = []

# Loop through all shapefiles
for file in shapefiles:
    # Read the shapefile as a GeoDataFrame and append to gdfs list
    gdfs.append(gpd.read_file(file))

# Concatenate all GeoDataFrames in the gdfs list
gdf = pd.concat(gdfs, ignore_index=True)

# Check if it worked
#gdf.info()

In [2]:
gdf.head()

,id,geometry
0,342472,POINT (8.66700 5.97800)
1,317280,POINT (105.48900 19.58300)
2,315528,POINT (-178.70000 -20.65000)
3,317204,POINT (103.00200 -4.33100)
4,303116,POINT (105.66300 17.98500)


In [3]:
gdf.rename(columns={'id': 'WDPAID'}, inplace=True)
gdf.head(2)

,WDPAID,geometry
0,342472,POINT (8.66700 5.97800)
1,317280,POINT (105.48900 19.58300)


### Merge CSVs and do some cleaning

In [ ]:
import glob
import pandas as pd

# Get a list of all CSV files in the directory
csv_files = glob.glob('*.csv')

# Specify the file you want to exclude
exclude_file = 'sub-continents.csv'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through all CSV files
for file in csv_files:
    # Check if the current file is the one to be excluded
    if file != exclude_file:
        try:
            # Read the CSV file as a DataFrame and append to dfs list
            dfs.append(pd.read_csv(file, low_memory=False))
        except pd.errors.EmptyDataError:
            print(f"Warning: {file} is empty and will be skipped.")

# Concatenate all DataFrames in the dfs list
df = pd.concat(dfs, ignore_index=True)

# Check if it worked for df
df.info()


In [ ]:
#df.head()

In [12]:
# Create a new DataFrame with 'ID' and 'ZSCOUNT' columns
sub_df = df[['WDPAID']].copy()
sub_df['ZSCOUNT'] = df.iloc[:, [31, 36, 39]].fillna(0).bfill(axis=1).iloc[:, 0]
sub_df['ZSSUM'] = df.iloc[:, [32, 37, 40]].fillna(0).bfill(axis=1).iloc[:, 0]
sub_df['ZSMEAN'] = df.iloc[:, [33, 38, 41]].fillna(0).bfill(axis=1).iloc[:, 0]


#print(sub_df)
#sub_df.head(-10)

In [13]:
mdf = pd.merge(df, sub_df, left_on='WDPAID', right_on='WDPAID', how='left')

In [ ]:
mdf.info()

In [15]:
# drop columns
mdf.drop(mdf.columns[[0, 5, 6, 10, 12, 13, 14, 16, 17, 18, 23, 26, 28, 29,
                    31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42]], axis=1, inplace=True)


In [16]:
missing_value_columns = mdf.columns[mdf.isna().any()].tolist()
print("Columns with missing values:", missing_value_columns)

Columns with missing values: ['NAME', 'MANG_AUTH']


In [17]:
# Print all rows where column x has missing values
rows_with_missing_values = mdf[mdf['NAME'].isna()][['NAME']]
print(rows_with_missing_values)

       NAME
244175  NaN


In [18]:
#Print the whole row
#row_244181 = mdf.iloc[244181, :]
#print(row_244181)

In [19]:
# Not in METADATAID from WDPAID_EU_feat.csv hence, 'not reported' is placed
mdf['MANG_AUTH'].fillna('not reported', inplace=True) 
# Checked the Name with the METADATAID and got the data title Padus 2.1; WDPAID_NA_feat.csv
mdf['NAME'].fillna('Padus 2.1', inplace=True) 

In [20]:
# Check for STATUS_YEAR missing data
zero_col = mdf.columns[(mdf == 0).any()].tolist()
print(zero_col)               # 0 in PA_DEF = OECM, Marine = terrestrial

['PA_DEF', 'MARINE', 'STATUS_YR', 'ZSCOUNT', 'ZSSUM', 'ZSMEAN']


In [21]:
# Count 0 in STATUS_YR
zero_count = mdf[mdf['STATUS_YR'] == 0]['STATUS_YR'].count()
print(zero_count)

31009


In [22]:
examples_0 = mdf[mdf['STATUS_YR'] == 0]['STATUS_YR'].head(2076)
#print("Values that are 0 in column STATUS_YR:\n", examples_0)

In [23]:
# Print the oldest year in STATUS_YR
second_oldest_year = df['STATUS_YR'].nsmallest(30997).iloc[-1]
#print(second_oldest_year)   # = 1800

filtyr = mdf.loc[mdf['STATUS_YR'] < 1800, 'STATUS_YR']

# Print the filtered 'STATUS_YR' values
#print(filtyr) # since 0 comes before 1800, it is confirmed to have nothing else

In [24]:
# Replace 0 with 1800 in STATUS_YR
mdf['STATUS_YR'] = mdf['STATUS_YR'].replace(0, 1700)

In [ ]:
mdf.info()

In [26]:
# Get rid of the decimals in WDPAID column
mdf['WDPAID'] = mdf['WDPAID'].astype(int)

In [ ]:
mdf.rename(columns={'perimeter': 'PERIMETER'}, inplace=True)
mdf.info()

In [28]:
# Splitting and taking the first entry of PARENT_ISO
mdf['PARENT_ISO'] = mdf['PARENT_ISO'].str.split(';').str[0]

#### Add Regions

In [ ]:
# read sub-continent df
cdf = pd.read_csv('sub-continents.csv')
cdf.head()

In [30]:
cdf.rename(columns={'alpha-3': 'PARENT_ISO'}, inplace=True)

In [ ]:
df = pd.merge(mdf, cdf, on='PARENT_ISO', how='left')
df.info()

In [32]:
# Drop columns
df.drop(df.columns[[22, 23, 24, 27, 28, 29, 30]], axis=1, inplace=True)

In [ ]:
df.rename(columns={'name': 'COUNTRY', 'region': 'CONTINENT', 'sub-region': 'REGION'}, inplace=True)
df.info()

In [34]:
missing_value_columns = df.columns[df.isna().any()].tolist()
print("Columns with missing values:", missing_value_columns)

Columns with missing values: []


### Merging df with gdf on WDPAID

In [ ]:
import pandas as pd
import geopandas as gpd

# Merge the DataFrame and GeoDataFrame on the 'id' column
point_m = df.merge(gdf, on='WDPAID', how='left')


point_m.info()

In [36]:
missing_value_columns = point_m.columns[point_m.isna().any()].tolist()
print("Columns with missing values:", missing_value_columns)

Columns with missing values: ['geometry']


In [ ]:
# Export point_merge.shp with all point geometries

# Create df1 with rows where 'geometry' has entries
df1 = point_m[point_m['geometry'].notnull()]
#df1.head()

# Export to CSV and SHP

In [42]:
#df1.to_csv('Point_final.csv', index=False)

# If 'geometry' column is present, convert df1 to a GeoDataFrame
#gdf1 = gpd.GeoDataFrame(df1, geometry='geometry')

# Export to Shapefile (SHP)
#gdf1.to_file('Point_final.shp')

In [39]:
# Export none_merge.shp with all none geometries in a next step to add the polygons

# Create df2 with rows where 'geometry' has no entries
df2 = point_m[point_m['geometry'].isnull()]
# Display the results
#df2.head()

,WDPAID,WDPA_PID,PA_DEF,NAME,DESIG_ENG,DESIG_TYPE,IUCN_CAT,MARINE,GIS_AREA,STATUS_YR,...,PARENT_ISO,CONS_OBJ,PERIMETER,ZSCOUNT,ZSSUM,ZSMEAN,COUNTRY,CONTINENT,REGION,geometry
0,555555520,555555520,1,Biliqo-Bulesa Community Conservancy,Community Nature Reserve,National,Not Reported,0,3812.709416,2007,...,KEN,Not Applicable,2.762080,0.239152,0.717455,3.0,Kenya,Africa,Sub-Saharan Africa,None
1,62222,62222,1,Lundazi Dam,Forest Reserve,National,Not Reported,0,4.582627,1966,...,ZMB,Not Applicable,0.094588,0.045825,0.091650,2.0,Zambia,Africa,Sub-Saharan Africa,None
2,40515,40515,1,Apac,Forest Reserve,National,Not Reported,0,0.055129,1965,...,UGA,Not Applicable,0.010716,0.000551,0.001103,2.0,Uganda,Africa,Sub-Saharan Africa,None
3,315106,315106,1,Bubulo,Forest Reserve,National,Not Reported,0,0.207022,1963,...,UGA,Not Applicable,0.017413,0.002070,0.004140,2.0,Uganda,Africa,Sub-Saharan Africa,None
4,40458,40458,1,Makoka,Forest Reserve,National,Not Reported,0,0.189273,1965,...,UGA,Not Applicable,0.017183,0.001893,0.003785,2.0,Uganda,Africa,Sub-Saharan Africa,None


In [43]:
# Export to CSV
#df2.to_csv('nopoly.csv', index=False)

# If 'geometry' column is present, convert df1 to a GeoDataFrame
#gdf2 = gpd.GeoDataFrame(df2, geometry='geometry')

# Export to Shapefile (SHP)
#gdf2.to_file('nopoly.shp')